In [259]:
import pandas as pd
import numpy as np
from torch import nn
import torch 
import matplotlib.pyplot as plt
from IPython.display import clear_output
import torch.utils.data as data_utils

In [260]:
line_dataframe = pd.read_csv('line_dataframe.csv')

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [261]:
df = line_dataframe
train, validate, test = np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])

In [262]:
train.shape, validate.shape, test.shape

((7921, 167), (2641, 167), (2641, 167))

In [263]:
features = ['week','year',
'line','linesag','linescrp','linemass','linedunk','linepfz','lineperry','linemore','linesea','linemars','lineray','lineargh','linecpa','linehow','linecoll','linegupt','lineed','lineharm','linefrez','linefly','linebihl','linemont','linetsr','lineavg','lineelo','linebill','linebmc','lineswt','lineborn','linem2','lineroth','linecpa2','linemaxy','linewolf','linebass','linepfz2','linesage','linesagp','linenyt','lineray2','linefox','lineopen','linepig','linetsr2','linecons','linematt','linekam','lineclaa','linecrain','linecong','linebmc2','linel2','linehank','linebenson','linestd','linelaz','lineald','linebemiss','linedoc','linemed','linemaas','lineash','linegupta','lineca','linesuper','lineimes','linecraw','lineburd','linesprs','linekeep','linebmore','linel2hf','linetsr3','linelog','linesagelo','linemarsee','linenutshell','linesagpred','linecparetro','linecovers','linethompson','linemassbcs','linemedian','lineanderson','linenutshellretro','linesportrends','linetsrslots','lineburdorf','linekerns','linetsrelo','lineatom','linesprs3','linetsr4','linetrexler','linedokter','linedwig','linegpa','linenatsportrank','linebill2','linegirl','linealder','linecather','lineschmidt','linepayne','lineaustin','linebythen','lineheresy','linetempo','linerba','linelaffaye','linepugh','linebf','linegrinder','linepiratings','linerwp','lineps','lineloud','linedoi','linepivint','linesid','linemidweek','linefeng','linefremeau','linepimean','linepibias','linecurry','linedonchess','linemassey','linemoore','linethocal','linethoavg','linesaggm','linenscombo','linethoats','linemds','linehorobik','linenseye','lineclean','lineespn','linepp','linecrunch','linesagr','linefpi','lineteamrank','linelabr','linebc2','lineround','linefei','linetalis','linejens','linepaynep','linepaynewl','lineharville','linegiddens','lineunion'
]

In [264]:
def make_feature_target(df):
    df = df[df['hscore'].notna()]
    df = df.fillna(-100)
    target_torch = torch.Tensor((df.hscore-df.vscore).to_numpy())
    scale = target_torch.std()
    target_torch = target_torch/scale
    
    features_torch = torch.Tensor(df[features].to_numpy())
    features_torch = features_torch/scale
    return features_torch, target_torch, scale
def make_naive_prediction(df):
    df = df[df['hscore'].notna()]
    df = df[df['line'].notna()]
    diff = (df.line-(df.hscore - df.vscore)).to_numpy()
    absError = np.mean(np.abs(diff))
    mse = np.mean(diff**2)**.5
    return mse,absError

In [266]:
train_feature, train_target, train_scale = make_feature_target(train)
test_feature, test_target, test_scale = make_feature_target(test)
mse_naive, absError_naive = make_naive_prediction(train)


train_feature.shape, mse_naive, absError_naive

(torch.Size([7916, 157]), 15.689921666130939, 12.352728884386476)

In [267]:
#.nn.BatchNorm1d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
batchNorm = nn.BatchNorm1d(157)

In [268]:
class Network(nn.Module):
    def __init__(self, nfeatures, p = .8):
        super().__init__()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p)
        self.batchnorm1 = nn.BatchNorm1d(157)
        self.batchnorm2 = nn.BatchNorm1d(20)
        self.batchnorm3 = nn.BatchNorm1d(10)
        self.hidden1 = nn.Linear(nfeatures, 20)
        self.hidden2 = nn.Linear(20, 10)
        self.hidden3 = nn.Linear(10, 10)
        self.hidden4 = nn.Linear(10, 10)
        self.hidden5 = nn.Linear(10, 10)
        self.hidden6 = nn.Linear(10, 10)
        self.hidden7 = nn.Linear(10, 1)
    def forward(self, x):
        x = self.batchnorm1(x)
        x = self.hidden1(x)
        #x = self.dropout(x)
        x = self.relu(x)
        x = self.hidden2(x)
        x = self.batchnorm3(x)
        #x = self.dropout(x)
        x = self.relu(x)
        x = self.hidden3(x)
        #x = self.dropout(x)
        x = self.relu(x)
        x = self.hidden4(x)
        #x = self.dropout(x)
        x = self.relu(x)
        x = self.hidden5(x)
        #x = self.dropout(x)
        x = self.relu(x)
        x = self.hidden6(x)
        #x = self.dropout(x)
        x = self.relu(x)
        x = self.hidden7(x)
        return x

In [269]:
spread_pred = Network(157, p=0.2)

In [270]:
lr = 1e-4
lossFunction = nn.MSELoss()
optimizer = torch.optim.Adam(spread_pred.parameters(),lr=lr)

In [273]:
train = data_utils.TensorDataset(train_feature, train_target)
trainloader = torch.utils.data.DataLoader(train, batch_size=100, shuffle=True)
#dataiter = iter(trainloader)

In [272]:
loss_array = []
loss_array_test = []
epochs = []
y_test = test_target.view(-1,1)
y = train_target.view(-1,1)

for i in range(100000):
    optimizer.zero_grad()
    ypred = spread_pred(train_feature)
    loss = lossFunction(ypred,y)
    loss.backward()
    optimizer.step()
    if i % 100 ==0:
        diff = ((ypred-y)*train_scale)
        mse = ((diff**2).mean())**.5
        loss_array.append(mse)
        epochs.append(i)

        ypred_test = spread_pred(test_feature)
        diff = (y_test-ypred_test)*test_scale
        mse_test = (diff**2).mean()**.5
        loss_array_test.append(mse_test)
        
        plt.plot(epochs,loss_array, label="train")
        plt.plot(epochs,loss_array_test, label="test")
        plt.plot([epochs[0],epochs[-1]],[mse_naive,mse_naive], label="naive")
        plt.legend()
        plt.show()
        clear_output(wait=True)

KeyboardInterrupt: 

In [274]:
loss_array = []
loss_array_test = []
epochs = []
y_test = test_target.view(-1,1)
y = train_target.view(-1,1)

for i in range(100000):
    #trainloader = torch.utils.data.DataLoader(train, batch_size=1000, shuffle=True)
    for feature, label in iter(trainloader):
        optimizer.zero_grad()
        y = label
        ypred = spread_pred(feature)
        loss = lossFunction(ypred,y)
        loss.backward()
        optimizer.step()
    if i % 1 ==0:
        diff = ((ypred-y)*train_scale)
        mse = ((diff**2).mean())**.5
        loss_array.append(mse)
        epochs.append(i)
        
        ypred_test = spread_pred(test_feature)
        diff = (y_test-ypred_test)*test_scale
        mse_test = (diff**2).mean()**.5
        loss_array_test.append(mse_test)
        
        plt.plot(epochs,loss_array, label="train")
        plt.plot(epochs,loss_array_test, label="test")
        plt.plot([epochs[0],epochs[-1]],[mse_naive,mse_naive], label="naive")
        plt.legend()
        plt.show()
        clear_output(wait=True)

KeyboardInterrupt: 

In [223]:

images, labels

(tensor([[ 0.2348, 94.5009,  0.5167,  ..., -4.6969, -4.6969, -4.6969],
         [ 0.1409, 94.5948, -1.5969,  ..., -4.6969, -4.6969, -4.6969],
         [ 0.1409, 94.2660,  0.5167,  ..., -4.6969, -4.6969, -4.6969],
         ...,
         [ 0.1879, 94.4070, -0.2114,  ..., -4.6969, -4.6969, -4.6969],
         [ 0.4697, 94.5009,  0.7750,  ..., -4.6969, -4.6969, -4.6969],
         [ 0.8924, 94.3130,  0.1174,  ..., -4.6969, -4.6969, -4.6969]]),
 tensor([ 0.7045, -1.9727,  0.1879,  0.3288, -0.7515,  0.5167,  0.3757,  0.3288,
         -3.2408,  1.4560, -0.1409,  0.1409,  0.8454,  0.1409, -0.3288,  0.9394,
          0.5636, -0.1879, -0.0470, -0.1409,  0.8924,  1.3151,  0.3288,  0.9863,
          0.5636, -0.2818, -0.6106,  0.2818,  1.5969,  1.5030,  0.5167,  0.0939,
         -1.1742, -0.3288,  1.9257,  0.7985, -0.0470,  1.4091, -0.8924, -1.1742,
          0.4697, -0.1879, -0.7045,  0.4697, -1.2212,  0.4697,  0.3288,  1.2682,
         -0.4697,  2.1136, -0.0470,  1.1272,  0.1879,  0.5636,  0.7985, 